# PA3 - Perceptron#

Zhanchong Deng  


A15491777  

#### Necessary Imports ####

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import Perceptron as pa3

#### Import Data ####

In [2]:
training = pa3.loadData('pa3train.txt')
test = pa3.loadData('pa3test.txt')
dictionary = pa3.loadDictionary('pa3dictionary.txt')
print(training.shape)
print(test.shape)
print(dictionary.shape)

(3000, 820)
(1000, 820)
(820,)


#### 1) Classify Class 1 or Class 2

Extract subset of training data with only class 1/2

In [3]:
has1_and2 = (training[:,-1] == 1) | (training[:,-1] == 2)
q1_training_set = training[has1_and2]
has1_and2 = (test[:,-1] == 1) | (test[:,-1] == 2)
q1_testing_set = test[has1_and2]

Training with 4 iterations

In [4]:
p = pa3.Perceptron()
p.fit(q1_training_set, 4)

Printing Errors

In [8]:
p.error(q1_testing_set, 1)

TypeError: predict_pass_one() missing 1 required positional argument: 'version'

In [5]:
for i in [2,3,4]:
    print("Training Error for", i, "run is:", p.error(q1_training_set, i))
    print("Testing Error for", i, "run is", p.error(q1_testing_set, i))
    print()

Training Error for 2 run is: 0.01834862385321101
Testing Error for 2 run is 0.04774535809018567

Training Error for 3 run is: 0.01834862385321101
Testing Error for 3 run is 0.04774535809018567

Training Error for 4 run is: 0.01834862385321101
Testing Error for 4 run is 0.04774535809018567



___

**5) Code from ID3.py:**  
```python
print("hi")
```